In [1]:
import math
from ipywidgets import interactive
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import numpy as np

In [2]:
# Fixed parameters
N = 21     # Number of participants
Tmax = 20  # total duration of the simulaion

# Tunable parameters with default values
Tinf = Tmax/10 # infectious time
I0 = 1         # number of initial cases
Itot = 0.8*N   # total number of cases
cr = 1/N       # per capita contact rate, 
               # C = cr x N is the number of contacts per unit of time an infectious individual make

In [3]:
# Using SciPy to specify the equations of the SIR model
# S'(t) = −beta * I * S
# I'(t) = beta * I S − gamma * I
# R'(t) = gamma * I
# S(0) = S0
# I(0) = I0
# R(0) = N - S0 - I0 = 0
# https://scipython.com/book/chapter-8-scipy/additional-examples/the-sir-epidemic-model/

# The SIR model differential equations.
def sir_equations(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I
    dIdt = beta * S * I - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [5]:
def plot_fun(Tinf=Tmax/10, I0=1, Itot=0.8*N, C=1):
    t = np.linspace(0, Tmax, Tmax)
    
    S0 = N - I0
    R0 = N - I0 - S0
    Send = N - Itot

    # z is gamma/beta
    z = Itot/math.log(S0/Send)

    BRN = S0/z
    print("Basic reproductive number =", BRN)

    Imax = N - z + z * math.log(z) - z * math.log(S0)
    print("Number of infections at the peak of outbreak =", Imax)

    gamma = 1/Tinf
    beta = gamma/z
    print("Transmission rate constant =", beta)

    cr = C/N
    p = beta/cr  # probability that a contact with a susceptible individual results in transmission
    print("Probability of infection per unit of time =", p)

    # Initial conditions vector
    y0 = S0, I0, R0
    # Integrate the SIR equations over the time grid, t.
    ret = odeint(sir_equations, y0, t, args=(N, beta, gamma))
    S, I, R = ret.T

    # Plot the data on three separate curves for S(t), I(t) and R(t)
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.plot(t, S, 'b', alpha=0.5, lw=2, label='Susceptible')
    ax.plot(t, I, 'r', alpha=0.5, lw=2, label='Infected')
    ax.plot(t, R, 'g', alpha=0.5, lw=2, label='Removed')
    ax.set_xlabel('Minutes')
    ax.set_ylabel('Number')
    ax.set_ylim(0, N)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(visible=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    plt.show()

interactive_plot = interactive(plot_fun, Tinf=(1.0, Tmax/2), I0=(1, int(N/10)), Itot=(1, N-1), C=(1, 10))
output = interactive_plot.children[-1]
output.layout.height = '700px'
interactive_plot

interactive(children=(FloatSlider(value=2.0, description='Tinf', max=10.0, min=1.0), IntSlider(value=1, descri…